In [5]:
import requests
import datetime
import pandas as pd
import numpy as np
import time

In [7]:
#define coins
coins_list = ["BTC",
"ETH",
"XRP",
"BCH",
"EOS",
"LTC",
"XLM",
"IOT",
"XMR",
"DASH",
"XEM",
"VEN",
"ETC",
"XVG",
"OMG",
"LSK",
"ZEC",
"BCN",
"BTS",
"SC",
"STRAT",
"DOGE",
"VERI",
"NEO",
"BTM*",
"STEEM",
"DGD",
"DCR",
"RHOC",
"WAVES",
"MKR",
"AE",
"SNT",
"KMD",
"REP",
"ARDR",
"ARK",
"DGB",
"PIVX",
"FCT",
"MONA",
"BNB",
"ZRX",
"LRC",
"HSR",
"WTC",
"SUB",
"VERI",
"NAS",
"GAS",
"FUN",
"SYS",
"KNC",
"GBYTE",
"SKY",
"RDD",
"NXT",
"XZC",
"MAID",
"BNT",
"STORJ",
"PAY",
"CND",
"EMC",
"ZEN",
"MAN",
"NXS",
"ICN",
"CVC",
"VTC",
"MANA",
"PART",
"BOS",
"GAME",
"UBQ",
"GNO",
"BLOCK",
"MTL",
"MCO",
"SAN",
"POE",
"BTX",
"ANT",
"CS",
"PLR",
"NAV",
"BTCD",
"ION",
"ADX",
"PPC",
"XBY",
"RLC",
"XDN",
"QRL",
"EDG",
"SLS",
"BAY"]

In [13]:
#functions to extract data from cryptocompare.com

def daily_price_historical(symbol, comparison_symbol='USD', limit=9999, aggregate=1, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    #df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    df['timestamp'] = df.time
    return df


def coins_daily_prices(list_of_coins, comparison_symbol='USD',limit=9999, aggregate = 1, exchange='',
                        price_type = 'close'):

    df = daily_price_historical(list_of_coins[0])
    df = df[['timestamp',price_type]]
    df.columns.values[1] ='{}_{}'.format(list_of_coins[0], price_type)
  
    for coin in list_of_coins[1:]:
        try:
            df1 = daily_price_historical(coin)
            df1 = df1[['timestamp',price_type]]
            df1.columns.values[1] ='{}_{}'.format(coin, price_type)
            df = pd.merge(df, df1, on='timestamp', how='outer')
        except:
            #check which currencies  names don't match
            print(coin)
            
    return(df)

def hourly_price_historical(symbol, comparison_symbol='USD', limit=9999, aggregate=1, exchange='', toTs = int(time.time())):
    url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}&toTs={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate, toTs)
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    #df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    df['timestamp'] = df.time
    return df


def coins_hourly_prices(list_of_coins, comparison_symbol='USD',limit=9999, aggregate = 1, exchange='', toTs = int(time.time()),
                        price_type = 'close'):

    df = hourly_price_historical(list_of_coins[0], 'USD', toTs = toTs)
    df = df[['timestamp',price_type]]
    df.columns.values[1] ='{}_{}'.format(list_of_coins[0], price_type)
  
    for coin in list_of_coins[1:]:
        try:
            df1 = hourly_price_historical(coin, comparison_symbol, limit = limit, aggregate = aggregate, toTs=toTs)
            df1 = df1[['timestamp',price_type]]
            df1.columns.values[1] ='{}_{}'.format(coin, price_type)
            df = pd.merge(df, df1, on='timestamp', how='outer')
        except:
            #check which currencies  names don't match
            print(coin)
            
    return(df)

def add_more_hourly_data(period1, coins, comparison_symbol='USD'):
    data_for_period2 = period1.timestamp.min()
    period2 = coins_hourly_prices(coins, toTs = data_for_period2, comparison_symbol=comparison_symbol)
    data = pd.concat([period2, period1], ignore_index=True)
    #data['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in data.timestamp]
    return data

def coins_daily_volatility(list_of_coins, comparison_symbol='USD',limit=9999, aggregate = 1, exchange=''):
    df = daily_price_historical(list_of_coins[0])
    df['volatility_of_log'] = 0.511*(np.log(df.high) - np.log(df.low))**2 - \
    0.019*((np.log(df.close)-np.log(df.open))*(np.log(df.high)+np.log(df.low)-2*np.log(df.open))- \
           2*(np.log(df.high)-np.log(df.open))*(np.log(df.low)-np.log(df.open)))-\
    0.383*(np.log(df.close)-np.log(df.open))**2
    df = df[['timestamp','volatility_of_log']]
    df.columns.values[1] ='{}'.format(list_of_coins[0])
  
    for coin in list_of_coins[1:]:
        try:
            df1 = daily_price_historical(coin)
            df1['volatility_of_log'] = 0.511*(np.log(df1.high) - np.log(df1.low))**2 - \
            0.019*((np.log(df1.close)-np.log(df1.open))*(np.log(df1.high)+np.log(df1.low)-2*np.log(df1.open))- \
                   2*(np.log(df1.high)-np.log(df1.open))*(np.log(df1.low)-np.log(df1.open)))-\
            0.383*(np.log(df1.close)-np.log(df1.open))**2
            df1 = df1[['timestamp','volatility_of_log']]
            df1.columns.values[1] ='{}'.format(coin)
            df = pd.merge(df, df1, on='timestamp', how='outer')
        except:
            #check which currencies  names don't match
            print(coin)
            
    return(df)

### Get daily prices

In [9]:
daily_return = coins_daily_prices(coins_list)
daily_return['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in daily_return.timestamp]

In [15]:
daily_return.head()

,timestamp,BTC_close,ETH_close,XRP_close,BCH_close,EOS_close,LTC_close,XLM_close,IOT_close,XMR_close,...,ION_close,ADX_close,PPC_close,XBY_close,RLC_close,XDN_close,QRL_close,EDG_close,SLS_close,BAY_close
0,2012-10-30 02:00:00,10.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2012-10-31 02:00:00,11.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2012-11-01 02:00:00,10.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2012-11-02 02:00:00,10.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2012-11-03 02:00:00,10.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
daily_return.shape

(2001, 98)

In [11]:
#save to csv
daily_return.to_csv('daily_return_USD.csv')

### Get daily volatility

In [14]:
daily_volatility = coins_daily_volatility(coins_list)

/Users/liza/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in log


In [15]:
daily_volatility['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in daily_volatility.timestamp]
daily_volatility.tail(20)

,timestamp,BTC,ETH,XRP,BCH,EOS,LTC,XLM,IOT,XMR,...,ION,ADX,PPC,XBY,RLC,XDN,QRL,EDG,SLS,BAY
1981,2018-04-03 03:00:00,0.001298,0.004168,0.002727,0.002602,0.001940,0.005534,0.001516,0.001324,0.001254,...,0.001430,0.001686,0.002370,0.002406,0.004204,0.016150,0.005180,0.004358,0.012201,0.002897
1982,2018-04-04 03:00:00,0.002158,0.003533,0.003428,0.002976,0.003473,0.005806,0.004930,0.005410,0.002240,...,0.002279,0.002107,0.003505,0.015848,0.009374,0.006404,0.002471,0.001864,0.006644,0.003526
1983,2018-04-05 03:00:00,0.001198,0.001450,0.001071,0.001773,0.009679,0.002774,0.003530,0.002586,0.001628,...,0.001742,0.000565,0.002804,0.008568,0.116995,0.004792,0.002756,0.003913,0.035414,0.000747
1984,2018-04-06 03:00:00,0.001104,0.001432,0.002376,0.001461,0.004437,0.001679,0.001174,0.002816,0.001918,...,0.000282,0.000403,0.007751,0.061586,0.000761,0.001277,0.000853,0.001394,0.000997,0.000333
1985,2018-04-07 03:00:00,0.001577,0.001990,0.001103,0.001707,0.000723,0.001490,0.001957,0.004651,0.001368,...,0.015587,0.000643,0.045577,0.035102,0.001695,0.002616,0.002088,0.000969,0.025162,0.004120
1986,2018-04-08 03:00:00,0.000356,0.000563,0.000346,0.000404,0.000391,0.000265,0.000679,0.002332,0.000415,...,0.019664,0.000376,0.000461,0.036178,0.002239,0.004825,0.011403,0.000821,0.013476,0.016762
1987,2018-04-09 03:00:00,0.002859,0.005614,0.002550,0.003633,0.001494,0.003561,0.005870,0.005056,0.003910,...,0.004644,0.029501,0.002567,0.023279,0.009526,0.006013,0.009597,0.006851,0.023690,0.010015
1988,2018-04-10 03:00:00,0.000602,0.001484,0.000489,0.001164,0.001343,0.000516,0.000737,0.001698,0.000373,...,0.000411,0.000865,0.002641,0.024123,0.003222,0.002743,0.000891,0.005526,0.003320,0.004619
1989,2018-04-11 03:00:00,0.000177,0.000943,0.005538,0.000171,0.031713,0.001161,0.001168,0.000990,0.000511,...,0.003769,0.000659,0.000621,0.010721,0.006281,0.021833,0.003084,0.001632,0.006855,0.010493
1990,2018-04-12 03:00:00,0.008135,0.009693,0.026557,0.003285,0.013340,0.007201,0.009076,0.027367,0.005241,...,0.001947,0.000953,0.011474,0.004539,0.007993,0.015318,0.003973,0.003237,0.013384,0.055237


In [16]:
daily_volatility.to_csv('daily_volatility_USD.csv')

### Get hourly return

In [17]:
hourly_return1 = coins_hourly_prices(coins_list)

In [19]:
hourly_return1.head()

,timestamp,BTC_close,ETH_close,XRP_close,BCH_close,EOS_close,LTC_close,XLM_close,IOT_close,XMR_close,...,ION_close,ADX_close,PPC_close,XBY_close,RLC_close,XDN_close,QRL_close,EDG_close,SLS_close,BAY_close
0,1517220000,11361.04,1182.11,1.29,1667.55,13.84,180.85,0.5940,2.43,319.62,...,4.99,1.92,5.07,0.2429,2.75,0.02672,2.24,1.33,73.80,0.1509
1,1517223600,11191.53,1164.95,1.26,1654.86,13.59,178.94,0.5770,2.41,315.71,...,4.90,1.87,5.01,0.2462,2.75,0.02621,2.19,1.30,72.48,0.1440
2,1517227200,11176.33,1165.43,1.27,1655.60,13.61,178.70,0.5785,2.42,319.36,...,4.85,1.86,4.96,0.2495,2.75,0.02649,2.18,1.29,70.84,0.1465
3,1517230800,11279.58,1176.98,1.29,1671.75,13.69,181.29,0.5811,2.42,321.12,...,5.01,1.89,4.79,0.2510,2.75,0.02673,2.22,1.30,71.51,0.1464
4,1517234400,11289.00,1180.43,1.29,1677.44,13.70,181.18,0.5799,2.43,319.18,...,5.02,1.91,4.82,0.2565,2.75,0.02704,2.16,1.29,72.34,0.1477


In [20]:
hourly_return2 = add_more_hourly_data(hourly_return1, coins_list)
hourly_return2.head()

,timestamp,BTC_close,ETH_close,XRP_close,BCH_close,EOS_close,LTC_close,XLM_close,IOT_close,XMR_close,...,ION_close,ADX_close,PPC_close,XBY_close,RLC_close,XDN_close,QRL_close,EDG_close,SLS_close,BAY_close
0,1510020000,7080.28,300.56,0.2019,598.12,0.9705,55.36,0.03197,0.3720,102.36,...,1.04,0.9456,1.31,0.01636,0.3011,0.002297,0.5899,0.5266,13.24,0.02287
1,1510023600,7126.93,298.73,0.2017,601.37,0.9787,55.38,0.03192,0.3779,101.76,...,1.04,0.9389,1.34,0.01646,0.3011,0.002297,0.5919,0.5267,14.25,0.02316
2,1510027200,7199.50,299.15,0.2015,610.74,0.9754,55.38,0.03078,0.3770,98.33,...,1.08,0.9746,1.35,0.01663,0.3011,0.002302,0.6014,0.5335,13.53,0.02297
3,1510030800,7128.29,297.50,0.2008,602.26,0.9582,55.21,0.03100,0.3771,97.15,...,1.07,0.9838,1.34,0.01647,0.3011,0.002302,0.5926,0.5256,13.55,0.02324
4,1510034400,7189.14,299.10,0.2010,603.36,0.9733,55.37,0.03146,0.3880,98.10,...,1.08,0.9694,1.35,0.01610,0.3011,0.002279,0.6008,0.5348,13.77,0.02257


In [22]:
hourly_return3 = add_more_hourly_data(hourly_return2, coins_list)
hourly_return3.shape

(6003, 98)

In [23]:
hourly_return3['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in hourly_return3.timestamp]
hourly_return3.describe()
hourly_return3.shape
hourly_return3.head()

,timestamp,BTC_close,ETH_close,XRP_close,BCH_close,EOS_close,LTC_close,XLM_close,IOT_close,XMR_close,...,ION_close,ADX_close,PPC_close,XBY_close,RLC_close,XDN_close,QRL_close,EDG_close,SLS_close,BAY_close
0,2017-08-15 21:00:00,4068.97,284.03,0.1574,294.27,1.62,42.75,0.01795,0.9196,47.96,...,1.97,0.5987,1.8,0.010580,0.0,0.002400,0.4163,0.8260,14.50,0.01945
1,2017-08-15 22:00:00,4057.59,283.79,0.1551,293.70,1.65,42.92,0.01784,0.8790,47.64,...,1.97,0.7807,1.8,0.009332,0.0,0.002421,0.4480,0.7888,15.01,0.01923
2,2017-08-15 23:00:00,4077.50,282.79,0.1560,298.38,1.65,43.30,0.01783,0.8816,47.89,...,2.09,0.8628,1.8,0.010600,0.0,0.002397,0.4461,0.7849,15.89,0.01921
3,2017-08-16 00:00:00,4133.36,284.66,0.1555,301.05,1.66,43.14,0.01773,0.9149,48.28,...,2.08,0.8494,1.8,0.009631,0.0,0.002348,0.4559,0.8721,15.67,0.01860
4,2017-08-16 01:00:00,4184.71,285.07,0.1569,300.46,1.64,43.54,0.01750,0.9149,48.84,...,1.91,0.9578,1.8,0.009834,0.0,0.002324,0.4783,0.8587,14.55,0.01946


In [24]:
hourly_return3.to_csv('hourly_return_USD.csv')